<a href="https://colab.research.google.com/github/ArshT/Pytorch_Practice/blob/master/MNIST_HP_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install optuna

     |████████████████████████████████| 296kB 16.6MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 163kB 26.4MB/s 
     |████████████████████████████████| 143kB 53.6MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 112kB 55.7MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=5e250249167c9221b2742effd147bd315da0f00518831294a7fa97a4bb63dc22
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [4]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
import optuna

In [5]:
def load_data():
  transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5),(0.5))])
  
  trainset = torchvision.datasets.MNIST(root='./data', train=True,download=True, transform=transform)
  
  testset = torchvision.datasets.MNIST(root='./data', train=False,download=True, transform=transform)

  trainloader = torch.utils.data.DataLoader(trainset,batch_size=32,shuffle = True,num_workers = 2)
  testloader = torch.utils.data.DataLoader(testset,batch_size=32,shuffle = False,num_workers = 2)

  return trainloader,testloader

In [36]:
class Net(nn.Module):

  def __init__(self,l1=512,l2=256):
    super(Net, self).__init__()

    self.drop = nn.Dropout(0.2)

    self.fc1 = nn.Linear(1*28*28,l1)
    self.fc2 = nn.Linear(l1,l2)
    self.fc3 = nn.Linear(l2,10)
  
  def forward(self,x):

    x = x.view(x.shape[0],1*28*28)

    x = F.relu(self.fc1(x))
    x = self.drop(x)
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [37]:
def train(log_interval,model,dataloader,epoch,optimizer,criterion):

  model.train()
  running_loss = 0.0
  for batch_idx, (data, target) in enumerate(dataloader):
    optimizer.zero_grad()

    data = data.to(torch.device('cuda:0'))
    target = target.to(torch.device('cuda:0'))

    output = model(data)
    loss = criterion(output,target)

    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if batch_idx % log_interval == log_interval-1:    
      print('[%d ,%5d] loss: %.3f' %(epoch, batch_idx + 1, running_loss / log_interval))
      running_loss = 0.0
      
def test(model, dataloader,criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataloader:
            data = data.to(torch.device('cuda:0'))
            target = target.to(torch.device('cuda:0'))
            
            output = model(data.to('cuda:0'))
            test_loss += criterion(output,target).item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.to('cuda:0').view_as(pred)).sum().item()

    test_loss /= len(dataloader.dataset)
    accuracy = 100. * correct / len(dataloader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(dataloader.dataset),100. * correct / len(dataloader.dataset)))
    
    return accuracy

In [42]:
def train_mnist(trial):

  cfg = { 'device' : "cuda" if torch.cuda.is_available() else "cpu",
          'n_epochs' : 10,
          'log_interval' : 300,
          'lr' : trial.suggest_loguniform('lr', 1e-4, 1e-1),          
          'l1' : trial.suggest_int('l1',16,512),
          'l2' : trial.suggest_int('l2',16,512)}



  train_loader, test_loader = load_data()
  print(cfg['l1'],cfg['l2'],cfg['lr'])
  model = Net(cfg['l1'],cfg['l2']).to('cuda:0')
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(),lr=cfg['lr'],momentum=0.95)
  
  for epoch in range(1, cfg['n_epochs'] + 1):
      train(cfg['log_interval'], model, train_loader,epoch,optimizer,criterion)
      test_accuracy = test(model, test_loader,criterion)
      print(test_accuracy)

  return test_accuracy

if __name__ == '__main__':

  sampler = optuna.samplers.TPESampler()
      
  study = optuna.create_study(sampler=sampler, direction='maximize')
  study.optimize(func=train_mnist, n_trials=100)
  #joblib.dump(study, '/content/gdrive/My Drive/Colab_Data/studies/mnist_optuna.pkl')

[I 2021-04-28 12:21:43,016] A new study created in memory with name: no-name-b7550949-791c-4574-82b3-5006347f5910


164 91 0.0001277743765567906
[1 ,  300] loss: 2.269
[1 ,  600] loss: 2.126
[1 ,  900] loss: 1.865
[1 , 1200] loss: 1.529
[1 , 1500] loss: 1.195
[1 , 1800] loss: 0.956

Test set: Average loss: 0.0233, Accuracy: 8187/10000 (82%)

81.87
[2 ,  300] loss: 0.757
[2 ,  600] loss: 0.673
[2 ,  900] loss: 0.627
[2 , 1200] loss: 0.565
[2 , 1500] loss: 0.560
[2 , 1800] loss: 0.523

Test set: Average loss: 0.0135, Accuracy: 8832/10000 (88%)

88.32
[3 ,  300] loss: 0.485
[3 ,  600] loss: 0.458
[3 ,  900] loss: 0.451
[3 , 1200] loss: 0.457
[3 , 1500] loss: 0.432
[3 , 1800] loss: 0.427

Test set: Average loss: 0.0111, Accuracy: 8993/10000 (90%)

89.93
[4 ,  300] loss: 0.408
[4 ,  600] loss: 0.413
[4 ,  900] loss: 0.383
[4 , 1200] loss: 0.385
[4 , 1500] loss: 0.359
[4 , 1800] loss: 0.375

Test set: Average loss: 0.0099, Accuracy: 9096/10000 (91%)

90.96
[5 ,  300] loss: 0.367
[5 ,  600] loss: 0.351
[5 ,  900] loss: 0.353
[5 , 1200] loss: 0.344
[5 , 1500] loss: 0.355
[5 , 1800] loss: 0.341

Test set: Av

[I 2021-04-28 12:23:47,198] Trial 0 finished with value: 93.96 and parameters: {'lr': 0.0001277743765567906, 'l1': 164, 'l2': 91}. Best is trial 0 with value: 93.96.



Test set: Average loss: 0.0066, Accuracy: 9396/10000 (94%)

93.96
187 403 0.0003431954533929123
[1 ,  300] loss: 2.097
[1 ,  600] loss: 1.206
[1 ,  900] loss: 0.717
[1 , 1200] loss: 0.565
[1 , 1500] loss: 0.495
[1 , 1800] loss: 0.449

Test set: Average loss: 0.0112, Accuracy: 8999/10000 (90%)

89.99
[2 ,  300] loss: 0.417
[2 ,  600] loss: 0.387
[2 ,  900] loss: 0.372
[2 , 1200] loss: 0.365
[2 , 1500] loss: 0.358
[2 , 1800] loss: 0.330

Test set: Average loss: 0.0086, Accuracy: 9208/10000 (92%)

92.08
[3 ,  300] loss: 0.317
[3 ,  600] loss: 0.323
[3 ,  900] loss: 0.297
[3 , 1200] loss: 0.275
[3 , 1500] loss: 0.285
[3 , 1800] loss: 0.286

Test set: Average loss: 0.0073, Accuracy: 9315/10000 (93%)

93.15
[4 ,  300] loss: 0.261
[4 ,  600] loss: 0.261
[4 ,  900] loss: 0.259
[4 , 1200] loss: 0.265
[4 , 1500] loss: 0.243
[4 , 1800] loss: 0.240

Test set: Average loss: 0.0065, Accuracy: 9377/10000 (94%)

93.77
[5 ,  300] loss: 0.227
[5 ,  600] loss: 0.227
[5 ,  900] loss: 0.225
[5 , 1200] los

[I 2021-04-28 12:25:52,485] Trial 1 finished with value: 96.32 and parameters: {'lr': 0.0003431954533929123, 'l1': 187, 'l2': 403}. Best is trial 1 with value: 96.32.


...


Test set: Average loss: 0.0045, Accuracy: 9588/10000 (96%)

95.88
340 392 0.002721189799036355
[1 ,  300] loss: 0.991
[1 ,  600] loss: 0.400
[1 ,  900] loss: 0.335
[1 , 1200] loss: 0.284
[1 , 1500] loss: 0.243
[1 , 1800] loss: 0.246

Test set: Average loss: 0.0054, Accuracy: 9473/10000 (95%)

94.73
[2 ,  300] loss: 0.189
[2 ,  600] loss: 0.190
[2 ,  900] loss: 0.192
[2 , 1200] loss: 0.163
[2 , 1500] loss: 0.164
[2 , 1800] loss: 0.164

Test set: Average loss: 0.0038, Accuracy: 9619/10000 (96%)

96.19
[3 ,  300] loss: 0.148
[3 ,  600] loss: 0.133
[3 ,  900] loss: 0.141
[3 , 1200] loss: 0.140
[3 , 1500] loss: 0.133
[3 , 1800] loss: 0.125

Test set: Average loss: 0.0031, Accuracy: 9690/10000 (97%)

96.9
[4 ,  300] loss: 0.112
[4 ,  600] loss: 0.122
[4 ,  900] loss: 0.121
[4 , 1200] loss: 0.107
[4 , 1500] loss: 0.104
[4 , 1800] loss: 0.101

Test set: Average loss: 0.0028, Accuracy: 9719/10000 (97%)

97.19
[5 ,  300] loss: 0.092
[5 ,  600] loss: 0.097
[5 ,  900] loss: 0.102
[5 , 1200] loss:

[I 2021-04-28 15:53:04,719] Trial 98 finished with value: 97.69 and parameters: {'lr': 0.002721189799036355, 'l1': 340, 'l2': 392}. Best is trial 83 with value: 98.18.



Test set: Average loss: 0.0023, Accuracy: 9769/10000 (98%)

97.69
493 326 0.006793997188593686
[1 ,  300] loss: 0.779
[1 ,  600] loss: 0.383
[1 ,  900] loss: 0.305
[1 , 1200] loss: 0.290
[1 , 1500] loss: 0.231
[1 , 1800] loss: 0.227

Test set: Average loss: 0.0062, Accuracy: 9345/10000 (93%)

93.45
[2 ,  300] loss: 0.193
[2 ,  600] loss: 0.183
[2 ,  900] loss: 0.187
[2 , 1200] loss: 0.171
[2 , 1500] loss: 0.182
[2 , 1800] loss: 0.172

Test set: Average loss: 0.0043, Accuracy: 9572/10000 (96%)

95.72
[3 ,  300] loss: 0.146
[3 ,  600] loss: 0.146
[3 ,  900] loss: 0.142
[3 , 1200] loss: 0.134
[3 , 1500] loss: 0.124
[3 , 1800] loss: 0.129

Test set: Average loss: 0.0034, Accuracy: 9643/10000 (96%)

96.43
[4 ,  300] loss: 0.107
[4 ,  600] loss: 0.122
[4 ,  900] loss: 0.118
[4 , 1200] loss: 0.124
[4 , 1500] loss: 0.110
[4 , 1800] loss: 0.104

Test set: Average loss: 0.0038, Accuracy: 9630/10000 (96%)

96.3
[5 ,  300] loss: 0.102
[5 ,  600] loss: 0.101
[5 ,  900] loss: 0.084
[5 , 1200] loss:

[I 2021-04-28 15:55:12,676] Trial 99 finished with value: 97.7 and parameters: {'lr': 0.006793997188593686, 'l1': 493, 'l2': 326}. Best is trial 83 with value: 98.18.



Test set: Average loss: 0.0025, Accuracy: 9770/10000 (98%)

97.7
